In [1]:
import errno
import matplotlib.pyplot as plt
import numpy as np
import cv2
import pandas as pd
from shapely.wkt import loads as wkt_loads
import tifffile as tiff
import os
import random
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K
from keras.models import load_model
from sklearn.metrics import jaccard_similarity_score
from shapely.geometry import MultiPolygon, Polygon
import shapely.wkt
import shapely.affinity
from collections import defaultdict

import csv
import glob
import gdal

from datetime import datetime

from keras import __version__
print(__version__)

Using TensorFlow backend.


2.0.5


In [2]:
def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

K.set_image_data_format('channels_first')

N_Cls = 1
data_base_dir = "/home/ubuntu/data/"
inDir = './'
mkdir_p(inDir + '/data')
out_dir = 'output/'
mkdir_p(out_dir)

print(os.getcwd())
DF = pd.read_csv(data_base_dir+'train_wkt_v4_TREES.csv')
val_DF = pd.read_csv(data_base_dir+'val_wkt_v4_TREES.csv')
GS = pd.read_csv(data_base_dir+'grid_sizes.csv', names=['ImageId', 'Xmax', 'Ymin'], skiprows=1)
SB = pd.read_csv(data_base_dir+'sample_submission.csv')

ISZ = 160
smooth = 1e-12

EPSILON = 2  # polygon edge smoothing factor (higher=less nodes, i.e. less detail)
MIN_AREA = 5.  # smallest area a polygon can be

val_img_names = ["6010_4_4", "6070_2_3", "6100_2_3", "6140_1_2", "6110_4_0"]

/home/ubuntu/git/kaggle-dstl/keras2-baseline-improvements


# Image preprocessing

In [24]:
def RGB(image_id):
    filename = os.path.join(data_base_dir, 'three_band', '{}.tif'.format(image_id))
    img = tiff.imread(filename)
    img = np.rollaxis(img, 0, 3)
    return img


def P(image_id):
    filename = os.path.join(data_base_dir, 'sixteen_band', '{}_P.tif'.format(image_id))
    img = tiff.imread(filename)
    img = img.reshape(img.shape[0], img.shape[1], 1)
    return img


#delete me:
def M(image_id):
    # __author__ = amaia
    # https://www.kaggle.com/aamaia/dstl-satellite-imagery-feature-detection/rgb-using-m-bands-example
    filename = os.path.join(data_base_dir, 'sixteen_band', '{}_M.tif'.format(image_id))
    img = tiff.imread(filename)
    img = np.rollaxis(img, 0, 3)
    return img


r=RGB(val_img_names[0])
print(r.shape)
m=M(val_img_names[0])
print(m.shape)

p=P(val_img_names[0])
print(p.shape)

(3345, 3393, 3)
(837, 848, 8)
(3348, 3392, 1)


Great - np.rollaxis still working!

In [4]:
def _convert_coordinates_to_raster(coords, img_size, xymax):
    # __author__ = visoft
    # https://www.kaggle.com/visoft/dstl-satellite-imagery-feature-detection/export-pixel-wise-mask
    Xmax, Ymax = xymax
    H, W = img_size
    W1 = 1.0 * W * W / (W + 1)
    H1 = 1.0 * H * H / (H + 1)
    xf = W1 / Xmax
    yf = H1 / Ymax
    coords[:, 1] *= yf
    coords[:, 0] *= xf
    coords_int = np.round(coords).astype(np.int32)
    return coords_int


def _get_xmax_ymin(grid_sizes_panda, imageId):
    # __author__ = visoft
    # https://www.kaggle.com/visoft/dstl-satellite-imagery-feature-detection/export-pixel-wise-mask
    xmax, ymin = grid_sizes_panda[grid_sizes_panda.ImageId == imageId].iloc[0, 1:].astype(float)
    return (xmax, ymin)


def _get_polygon_list(wkt_list_pandas, imageId, cType):
    # __author__ = visoft
    # https://www.kaggle.com/visoft/dstl-satellite-imagery-feature-detection/export-pixel-wise-mask
    df_image = wkt_list_pandas[wkt_list_pandas.ImageId == imageId]
    multipoly_def = df_image[df_image.ClassType == cType].MultipolygonWKT
    polygonList = None
    if len(multipoly_def) > 0:
        assert len(multipoly_def) == 1
        polygonList = wkt_loads(multipoly_def.values[0])
    return polygonList


def _get_and_convert_contours(polygonList, raster_img_size, xymax):
    # __author__ = visoft
    # https://www.kaggle.com/visoft/dstl-satellite-imagery-feature-detection/export-pixel-wise-mask
    perim_list = []
    interior_list = []
    if polygonList is None:
        return None
    for k in range(len(polygonList)):
        poly = polygonList[k]
        perim = np.array(list(poly.exterior.coords))
        perim_c = _convert_coordinates_to_raster(perim, raster_img_size, xymax)
        perim_list.append(perim_c)
        for pi in poly.interiors:
            interior = np.array(list(pi.coords))
            interior_c = _convert_coordinates_to_raster(interior, raster_img_size, xymax)
            interior_list.append(interior_c)
    return perim_list, interior_list


def _plot_mask_from_contours(raster_img_size, contours, class_value=1):
    # __author__ = visoft
    # https://www.kaggle.com/visoft/dstl-satellite-imagery-feature-detection/export-pixel-wise-mask
    img_mask = np.zeros(raster_img_size, np.uint8)
    if contours is None:
        return img_mask
    perim_list, interior_list = contours
    cv2.fillPoly(img_mask, perim_list, class_value)
    cv2.fillPoly(img_mask, interior_list, 0)
    return img_mask


def generate_mask_for_image_and_class(raster_size, imageId, class_type, grid_sizes_panda=GS, wkt_list_pandas=DF):
    # __author__ = visoft
    # https://www.kaggle.com/visoft/dstl-satellite-imagery-feature-detection/export-pixel-wise-mask
    xymax = _get_xmax_ymin(grid_sizes_panda, imageId)  # for scaling according to competition
    polygon_list = _get_polygon_list(wkt_list_pandas, imageId, class_type)  # read (training) polygon data
    contours = _get_and_convert_contours(polygon_list, raster_size, xymax)  # creating outline from vector nodes
    mask = _plot_mask_from_contours(raster_size, contours, 1)  # filling in polygon outlines (i.e. creating the mask)
    return mask


# def M(image_id):
#     # __author__ = amaia
#     # https://www.kaggle.com/aamaia/dstl-satellite-imagery-feature-detection/rgb-using-m-bands-example
#     filename = os.path.join(data_base_dir, 'sixteen_band', '{}_M.tif'.format(image_id))
#     img = tiff.imread(filename)
#     img = np.rollaxis(img, 0, 3)
#     return img


def stretch_n(bands, lower_percent=0, higher_percent=100): # <- "Claims to improve" (was lower_percent=5, higher_percent=95)
    # "Contrast enhancement", see https://www.kaggle.com/aamaia/rgb-using-m-bands-example
    out = np.zeros_like(bands).astype(np.float32)
    n = bands.shape[2]
    for i in range(n):
        a = 0  # np.min(band)
        b = 1  # np.max(band)
        c = np.percentile(bands[:, :, i], lower_percent)
        d = np.percentile(bands[:, :, i], higher_percent)
        t = a + (bands[:, :, i] - c) * (b - a) / (d - c)
        t[t < a] = a
        t[t > b] = b
        out[:, :, i] = t

    return out.astype(np.float32)

    
def get_patches(img, msk, amt=10000, aug=True):
    is2 = int(1.0 * ISZ)  # ISZ=is2=patch size
    xm, ym = img.shape[0] - is2, img.shape[1] - is2  # starting coordinate for getting patches (taking into consideration padding)

    x, y = [], []

    # threshold determining when to add an image - if the image contains a greater percentage of pixels in the training
    # set than specified by the thresholds below:
    tr = [0.3]
    
    count = 0

    for i in range(amt):
        do_we_append = False
        
        # get random point in inner image (i.e exluding the outer image padding)
        xc = random.randint(0, xm)
        yc = random.randint(0, ym)

        # get patch
        im = img[xc:xc + is2, yc:yc + is2]
        ms = msk[xc:xc + is2, yc:yc + is2]

        for j in range(N_Cls):
            # sum pixels containing class j
            sm = np.sum(ms[:, :, j])
            if aug:
                if random.uniform(0, 1) > 0.5:
                    im = im[::-1]
                    ms = ms[::-1]
                if random.uniform(0, 1) > 0.5:
                    im = im[:, ::-1]
                    ms = ms[:, ::-1]

            # calculate the percent of covered pixels (for one class) - check if greater than threshold.
            if 1.0 * sm / is2 ** 2 > tr[j]:
                do_we_append = True
            
            # Add "blank ones" with probability 0.1
            elif sm < 500:
                do_we_append = True
                count += 1
                print("Add blank, idx={}, count={}".format(i, count))
            
        if do_we_append:
            x.append(im)
            y.append(ms)

    x, y = 2 * np.transpose(x, (0, 3, 1, 2)) - 1, np.transpose(y, (0, 3, 1, 2))  # Maybe some sort of standardisation/normalisation
    print(x.shape, y.shape, np.amax(x), np.amin(x), np.amax(y), np.amin(y))
    return x, y

In [5]:
def jaccard_coef(y_true, y_pred):
    # __author__ = Vladimir Iglovikov
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_int(y_true, y_pred):
    # __author__ = Vladimir Iglovikov
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))

    intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return K.mean(jac)


def calc_jacc(model):
    img = np.load(inDir + '/data/x_val_%d.npy' % N_Cls)
    msk = np.load(inDir + '/data/y_val_%d.npy' % N_Cls)
    
    print("Validation data shapes:")
    print(img.shape)
    print(msk.shape)


    prd = model.predict(img, batch_size=4)
    print(prd.shape, msk.shape)
    avg, trs = [], []

    for i in range(N_Cls):
        t_msk = msk[:, i, :, :]
        t_prd = prd[:, i, :, :]
        t_msk = t_msk.reshape(msk.shape[0] * msk.shape[2], msk.shape[3])
        t_prd = t_prd.reshape(msk.shape[0] * msk.shape[2], msk.shape[3])

        # grid search the threshold
        m, b_tr = 0, 0
        for j in range(10):
            tr = j / 10.0  # threshold
            pred_binary_mask = t_prd > tr

            jk = jaccard_similarity_score(t_msk, pred_binary_mask)
            if jk > m:
                m = jk
                b_tr = tr

        print(i, m, b_tr)
        avg.append(m)
        trs.append(b_tr)

    score = sum(avg) / 10.0
    return prd, score, trs  # trs is the best threshold value (a list, for each of the 10 classes)

# RGB

In [13]:
def make_val(aug=False):
    print("let's pick some samples for validation")
    img = np.load(inDir + '/data/x_valALL_%d.npy' % N_Cls)
    msk = np.load(inDir + '/data/y_valALL_%d.npy' % N_Cls)
    x, y = get_patches(img, msk, amt=100, aug=aug)  # amt := attempt (maybe - i.e. attempting to create 3000 patches, without
                                            # percentage area cover threshold.

    print("Validation data shapes:")
    print(x.shape)
    print(y.shape)

    np.save(inDir + '/data/x_val_%d' % N_Cls, x)
    np.save(inDir + '/data/y_val_%d' % N_Cls, y)
    
    
def stick_all_train():
    print("let's stick all imgs together")
    s = 3300  # size of the RGB images (roughly)

    # 25 training images grid
    x = np.zeros((5 * s, 4 * s, 3))      # RGB
    y = np.zeros((5 * s, 4 * s, N_Cls))  # axis=2 denote the class label

    ids = sorted(DF.ImageId.unique())
    print(len(ids))
    
    
    # x start grid position (based on image size)
    for i in range(5):
        # y start grid position (based on image size)
        for j in range(4):
            id = ids[4 * i + j]

            img = RGB(id)
        
            img = stretch_n(img)
            print(img.shape, id, np.amax(img), np.amin(img))
            
            x[s * i:s * i + s, s * j:s * j + s, :] = img[:s, :s, :]
            
            # generate training masks by class
            for z in range(N_Cls):
                y[s * i:s * i + s, s * j:s * j + s, z] = generate_mask_for_image_and_class(
                    (img.shape[0], img.shape[1]), id, z + 1)[:s, :s]
    
    print("x shape is:")
    print(x.shape)
    print("y shape is:")
    print(y.shape)

    print(np.amax(y), np.amin(y))
    
    np.save(inDir + '/data/x_trn_%d' % N_Cls, x)
    np.save(inDir + '/data/y_trn_%d' % N_Cls, y)
    
    
def stick_all_val():
    print("let's stick all imgs together")
    s = 3300  # size of the RG images (roughly)

    # 25 training images grid
    x = np.zeros((5 * s, 1 * s, 3))      # RGB
    y = np.zeros((5 * s, 1 * s, N_Cls))   # axis=2 denote the class label

    ids = sorted(val_DF.ImageId.unique())
    print(len(ids))

    # x start grid position (based on image size)
    for i in range(5):
        # y start grid position (based on image size)
        for j in range(1):
            id = ids[1 * i + j]

            img = RGB(id)
            
            img = stretch_n(img)
            print(img.shape, id, np.amax(img), np.amin(img))
            
            x[s * i:s * i + s, s * j:s * j + s, :] = img[:s, :s, :]  
            
            # generate training masks by class
            for z in range(N_Cls):
                y[s * i:s * i + s, s * j:s * j + s, z] = generate_mask_for_image_and_class(
                    (img.shape[0], img.shape[1]), id, z + 1, wkt_list_pandas=val_DF)[:s, :s]

    print("x shape is:")
    print(x.shape)
    print("y shape is:")
    print(y.shape)

    print(np.amax(y), np.amin(y))

    np.save(inDir + '/data/x_valALL_%d' % N_Cls, x)
    np.save(inDir + '/data/y_valALL_%d' % N_Cls, y)

In [14]:
stick_all_train()

stick_all_val()
make_val()

let's stick all imgs together
20
(3349, 3396, 3) 6010_1_2 1.0 0.0
(3345, 3396, 3) 6010_4_2 1.0 0.0
(3349, 3391, 3) 6040_1_0 1.0 0.0
(3349, 3391, 3) 6040_1_3 1.0 0.0
(3349, 3391, 3) 6040_2_2 1.0 0.0
(3346, 3387, 3) 6040_4_4 1.0 0.0
(3348, 3403, 3) 6060_2_3 1.0 0.0
(3349, 3393, 3) 6090_2_0 1.0 0.0
(3349, 3391, 3) 6100_1_3 1.0 0.0
(3349, 3391, 3) 6100_2_2 1.0 0.0
(3348, 3396, 3) 6110_1_2 1.0 0.0
(3348, 3396, 3) 6110_3_1 1.0 0.0
(3348, 3396, 3) 6110_4_0 1.0 0.0
(3348, 3403, 3) 6120_2_0 1.0 0.0
(3348, 3403, 3) 6120_2_2 1.0 0.0
(3348, 3396, 3) 6140_3_1 1.0 0.0
(3348, 3403, 3) 6150_2_3 1.0 0.0
(3349, 3393, 3) 6160_2_1 1.0 0.0
(3349, 3389, 3) 6170_0_4 1.0 0.0
(3345, 3393, 3) 6170_4_1 1.0 0.0
x shape is:
(16500, 13200, 3)
y shape is:
(16500, 13200, 1)
1.0 0.0
let's stick all imgs together
5
(3345, 3393, 3) 6010_4_4 1.0 0.0
(3350, 3338, 3) 6070_2_3 1.0 0.0
(3349, 3391, 3) 6100_2_3 1.0 0.0
(3348, 3396, 3) 6140_1_2 1.0 0.0
(3349, 3389, 3) 6170_2_4 1.0 0.0
x shape is:
(16500, 3300, 3)
y shape is:
(

In [15]:
def get_unet_RGB():
    inputs = Input((3, ISZ, ISZ))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=1)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=1)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=1)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=1)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(N_Cls, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])
    model.compile(optimizer=Adam(), loss='binary_crossentropy', 
                  metrics=[jaccard_coef, jaccard_coef_int, 'accuracy'])
    return model

In [16]:
x_val, y_val = np.load(inDir + '/data/x_val_%d.npy' % N_Cls), np.load(inDir + '/data/y_val_%d.npy' % N_Cls)
img = np.load(inDir + '/data/x_trn_%d.npy' % N_Cls)
msk = np.load(inDir + '/data/y_trn_%d.npy' % N_Cls)

model = get_unet_RGB()

# del x_trn
# del y_trn
x_trn, y_trn = get_patches(img, msk, amt=200, aug=False)

Add blank, idx=2, count=1
Add blank, idx=12, count=2
Add blank, idx=16, count=3
Add blank, idx=17, count=4
Add blank, idx=19, count=5
Add blank, idx=20, count=6
Add blank, idx=21, count=7
Add blank, idx=26, count=8
Add blank, idx=32, count=9
Add blank, idx=40, count=10
Add blank, idx=42, count=11
Add blank, idx=50, count=12
Add blank, idx=53, count=13
Add blank, idx=55, count=14
Add blank, idx=60, count=15
Add blank, idx=62, count=16
Add blank, idx=65, count=17
Add blank, idx=74, count=18
Add blank, idx=82, count=19
Add blank, idx=88, count=20
Add blank, idx=92, count=21
Add blank, idx=95, count=22
Add blank, idx=111, count=23
Add blank, idx=113, count=24
Add blank, idx=128, count=25
Add blank, idx=134, count=26
Add blank, idx=135, count=27
Add blank, idx=141, count=28
Add blank, idx=143, count=29
Add blank, idx=146, count=30
Add blank, idx=148, count=31
Add blank, idx=149, count=32
Add blank, idx=151, count=33
Add blank, idx=152, count=34
Add blank, idx=157, count=35
Add blank, idx=15

In [17]:
print(x_trn.shape)
print(x_val.shape)

(54, 3, 160, 160)
(52, 3, 160, 160)


### Quick check trn/val images look sensible (to prevent massively different/overfitting)

In [12]:
import scipy.misc

mkdir_p("visualise/trn")
mkdir_p("visualise/val")

def save_visual_msk_patch(np_arr, folder, sample_idx=0):
    scipy.misc.imsave("visualise/{}/{}msk.bmp".format(folder, sample_idx), np_arr[sample_idx][0])

def save_visual_img_patch(np_arr, folder, sample_idx=0, band=0):
    scipy.misc.imsave("visualise/{}/{}img_{}.bmp".format(folder, sample_idx, band), np_arr[sample_idx][0])

In [13]:
for i in range(x_trn.shape[0]):
    save_visual_img_patch(x_trn, "trn", sample_idx=i)
    save_visual_msk_patch(y_trn, "trn", sample_idx=i)
    
for i in range(x_val.shape[0]):
    save_visual_img_patch(x_val, "val", sample_idx=i)
    save_visual_msk_patch(y_val, "val", sample_idx=i)

Looks good - resolution definately better!

## Training

In [18]:
K.set_value(model.optimizer.lr, 0.1)

model.fit(x_trn, y_trn, batch_size=64, epochs=1, verbose=1, shuffle=True,
          validation_data=(x_val, y_val))

Train on 54 samples, validate on 52 samples
Epoch 1/1
54/54 [==============================] - 129s - loss: 0.6829 - jaccard_coef: 0.0738 - jaccard_coef_int: 0.0056 - acc: 0.8109 - val_loss: 2.0064 - val_jaccard_coef: 6.0346e-18 - val_jaccard_coef_int: 6.0346e-18 - val_acc: 0.8755


In [25]:
K.set_value(model.optimizer.lr, 0.1)

model.fit(x_trn, y_trn, batch_size=64, epochs=1, verbose=1, shuffle=True,
          validation_data=(x_val, y_val))

Train on 54 samples, validate on 52 samples
Epoch 1/1
54/54 [==============================] - 128s - loss: 1.2886 - jaccard_coef: 9.0485e-18 - jaccard_coef_int: 9.0485e-18 - acc: 0.9201 - val_loss: 2.0064 - val_jaccard_coef: 6.0346e-18 - val_jaccard_coef_int: 6.0346e-18 - val_acc: 0.8755


In [26]:
K.set_value(model.optimizer.lr, 0.01)

model.fit(x_trn, y_trn, batch_size=64, epochs=1, verbose=1, shuffle=True,
          validation_data=(x_val, y_val))

Train on 54 samples, validate on 52 samples
Epoch 1/1
54/54 [==============================] - 128s - loss: 1.2886 - jaccard_coef: 9.0485e-18 - jaccard_coef_int: 9.0485e-18 - acc: 0.9201 - val_loss: 2.0064 - val_jaccard_coef: 6.0346e-18 - val_jaccard_coef_int: 6.0346e-18 - val_acc: 0.8755


In [27]:
K.set_value(model.optimizer.lr, 0.0001)

model.fit(x_trn, y_trn, batch_size=64, epochs=1, verbose=1, shuffle=True,
          validation_data=(x_val, y_val))

Train on 54 samples, validate on 52 samples
Epoch 1/1
54/54 [==============================] - 128s - loss: 1.2886 - jaccard_coef: 9.0485e-18 - jaccard_coef_int: 9.0485e-18 - acc: 0.9201 - val_loss: 2.0064 - val_jaccard_coef: 6.0346e-18 - val_jaccard_coef_int: 6.0346e-18 - val_acc: 0.8755


Hmmm.. possibly some preprocessing doesn't work for the big images that work for M band needs for investigation.

# P band

In [ ]:
def make_val(aug=False):
    print("let's pick some samples for validation")
    img = np.load(inDir + '/data/x_valALL_%d.npy' % N_Cls)
    msk = np.load(inDir + '/data/y_valALL_%d.npy' % N_Cls)
    x, y = get_patches(img, msk, amt=100, aug=aug)  # amt := attempt (maybe - i.e. attempting to create 3000 patches, without
                                            # percentage area cover threshold.

    print("Validation data shapes:")
    print(x.shape)
    print(y.shape)

    np.save(inDir + '/data/x_val_%d' % N_Cls, x)
    np.save(inDir + '/data/y_val_%d' % N_Cls, y)
    
    
def stick_all_train():
    print("let's stick all imgs together")
    s = 3300  # size of the P images (roughly)

    # 25 training images grid
    x = np.zeros((5 * s, 4 * s, 1))      # P
    y = np.zeros((5 * s, 4 * s, N_Cls))  # axis=2 denote the class label

    ids = sorted(DF.ImageId.unique())
    print(len(ids))
    
    
    # x start grid position (based on image size)
    for i in range(5):
        # y start grid position (based on image size)
        for j in range(4):
            id = ids[4 * i + j]

            img = P(id)
        
            img = stretch_n(img)
            print(img.shape, id, np.amax(img), np.amin(img))
            
            x[s * i:s * i + s, s * j:s * j + s, :] = img[:s, :s, :]
            
            # generate training masks by class
            for z in range(N_Cls):
                y[s * i:s * i + s, s * j:s * j + s, z] = generate_mask_for_image_and_class(
                    (img.shape[0], img.shape[1]), id, z + 1)[:s, :s]
    
    print("x shape is:")
    print(x.shape)
    print("y shape is:")
    print(y.shape)

    print(np.amax(y), np.amin(y))
    
    np.save(inDir + '/data/x_trn_%d' % N_Cls, x)
    np.save(inDir + '/data/y_trn_%d' % N_Cls, y)
    
    
def stick_all_val():
    print("let's stick all imgs together")
    s = 3300  # size of the P images (roughly)

    # 25 training images grid
    x = np.zeros((5 * s, 1 * s, 1))      # P
    y = np.zeros((5 * s, 1 * s, N_Cls))   # axis=2 denote the class label

    ids = sorted(val_DF.ImageId.unique())
    print(len(ids))

    # x start grid position (based on image size)
    for i in range(5):
        # y start grid position (based on image size)
        for j in range(1):
            id = ids[1 * i + j]

            img = P(id)
            
            img = stretch_n(img)
            print(img.shape, id, np.amax(img), np.amin(img))
            
            x[s * i:s * i + s, s * j:s * j + s, :] = img[:s, :s, :]  
            
            # generate training masks by class
            for z in range(N_Cls):
                y[s * i:s * i + s, s * j:s * j + s, z] = generate_mask_for_image_and_class(
                    (img.shape[0], img.shape[1]), id, z + 1, wkt_list_pandas=val_DF)[:s, :s]

    print("x shape is:")
    print(x.shape)
    print("y shape is:")
    print(y.shape)

    print(np.amax(y), np.amin(y))

    np.save(inDir + '/data/x_valALL_%d' % N_Cls, x)
    np.save(inDir + '/data/y_valALL_%d' % N_Cls, y)

In [ ]:
stick_all_train()

stick_all_val()
make_val()

In [ ]:
def get_unet_P():
    inputs = Input((1, ISZ, ISZ))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=1)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=1)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=1)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=1)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(N_Cls, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])
    model.compile(optimizer=Adam(), loss='binary_crossentropy', 
                  metrics=[jaccard_coef, jaccard_coef_int, 'accuracy'])
    return model

In [ ]:
x_val, y_val = np.load(inDir + '/data/x_val_%d.npy' % N_Cls), np.load(inDir + '/data/y_val_%d.npy' % N_Cls)
img = np.load(inDir + '/data/x_trn_%d.npy' % N_Cls)
msk = np.load(inDir + '/data/y_trn_%d.npy' % N_Cls)

model = get_unet_P()

# del x_trn
# del y_trn
x_trn, y_trn = get_patches(img, msk, amt=200, aug=False)

## Training

In [ ]:
K.set_value(model.optimizer.lr, 0.1)

model.fit(x_trn, y_trn, batch_size=64, epochs=1, verbose=1, shuffle=True,
          validation_data=(x_val, y_val))

# RGB+P band

In [ ]:
def make_val(aug=False):
    print("let's pick some samples for validation")
    img = np.load(inDir + '/data/x_valALL_%d.npy' % N_Cls)
    msk = np.load(inDir + '/data/y_valALL_%d.npy' % N_Cls)
    x, y = get_patches(img, msk, amt=100, aug=aug)  # amt := attempt (maybe - i.e. attempting to create 3000 patches, without
                                            # percentage area cover threshold.

    print("Validation data shapes:")
    print(x.shape)
    print(y.shape)

    np.save(inDir + '/data/x_val_%d' % N_Cls, x)
    np.save(inDir + '/data/y_val_%d' % N_Cls, y)
    
    
def stick_all_train():
    print("let's stick all imgs together")
    s = 3300  # size of the P images (roughly)

    # 25 training images grid
    x = np.zeros((5 * s, 4 * s, 3+1))      # RGB+P
    y = np.zeros((5 * s, 4 * s, N_Cls))  # axis=2 denote the class label

    ids = sorted(DF.ImageId.unique())
    print(len(ids))
    
    
    # x start grid position (based on image size)
    for i in range(5):
        # y start grid position (based on image size)
        for j in range(4):
            id = ids[4 * i + j]

            img = RGB(id)
            img_p = P(id)
            
            img = stretch_n(img)
            img_p = stretch_n(img_p)
            
            print(img.shape, id, np.amax(img), np.amin(img))
            
            x[s * i:s * i + s, s * j:s * j + s, :3] = img[:s, :s, :]  
            x[s * i:s * i + s, s * j:s * j + s, 3] = img_p[:s, :s, :] 
            
            # generate training masks by class
            for z in range(N_Cls):
                y[s * i:s * i + s, s * j:s * j + s, z] = generate_mask_for_image_and_class(
                    (img.shape[0], img.shape[1]), id, z + 1)[:s, :s]
    
    print("x shape is:")
    print(x.shape)
    print("y shape is:")
    print(y.shape)

    print(np.amax(y), np.amin(y))
    
    np.save(inDir + '/data/x_trn_%d' % N_Cls, x)
    np.save(inDir + '/data/y_trn_%d' % N_Cls, y)
    
    
def stick_all_val():
    print("let's stick all imgs together")
    s = 3300  # size of the P images (roughly)

    # 25 training images grid
    x = np.zeros((5 * s, 1 * s, 3+1))      # RGB+P
    y = np.zeros((5 * s, 1 * s, N_Cls))   # axis=2 denote the class label

    ids = sorted(val_DF.ImageId.unique())
    print(len(ids))

    # x start grid position (based on image size)
    for i in range(5):
        # y start grid position (based on image size)
        for j in range(1):
            id = ids[1 * i + j]

            img = RGB(id)
            img_p = P(id)
            
            img = stretch_n(img)
            img_p = stretch_n(img_p)
            
            print(img.shape, id, np.amax(img), np.amin(img))
            
            x[s * i:s * i + s, s * j:s * j + s, :3] = img[:s, :s, :]  
            x[s * i:s * i + s, s * j:s * j + s, 3] = img_p[:s, :s, :]  
            
            # generate training masks by class
            for z in range(N_Cls):
                y[s * i:s * i + s, s * j:s * j + s, z] = generate_mask_for_image_and_class(
                    (img.shape[0], img.shape[1]), id, z + 1, wkt_list_pandas=val_DF)[:s, :s]

    print("x shape is:")
    print(x.shape)
    print("y shape is:")
    print(y.shape)

    print(np.amax(y), np.amin(y))

    np.save(inDir + '/data/x_valALL_%d' % N_Cls, x)
    np.save(inDir + '/data/y_valALL_%d' % N_Cls, y)

In [ ]:
stick_all_train()

stick_all_val()
make_val()

In [ ]:
def get_unet_RGBP():
    inputs = Input((3+1, ISZ, ISZ))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=1)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=1)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=1)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=1)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(N_Cls, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])
    model.compile(optimizer=Adam(), loss='binary_crossentropy', 
                  metrics=[jaccard_coef, jaccard_coef_int, 'accuracy'])
    return model

In [ ]:
x_val, y_val = np.load(inDir + '/data/x_val_%d.npy' % N_Cls), np.load(inDir + '/data/y_val_%d.npy' % N_Cls)
img = np.load(inDir + '/data/x_trn_%d.npy' % N_Cls)
msk = np.load(inDir + '/data/y_trn_%d.npy' % N_Cls)

model = get_unet_RGBP()

# del x_trn
# del y_trn
x_trn, y_trn = get_patches(img, msk, amt=200, aug=False)

## Training

In [ ]:
K.set_value(model.optimizer.lr, 0.01)

model.fit(x_trn, y_trn, batch_size=64, epochs=1, verbose=1, shuffle=True,
          validation_data=(x_val, y_val))